In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Define our coupled derivatives to integrate

In [3]:
def dydx(x,y):
    
    #set the derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #dydx=z
    #dzdx=-y
    
    #set y=y[0]
    #set z=y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx=z
    y_derivs[0] = y[1]
    
    #set dzdx=-y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs

# Define the 4th order RK method

In [4]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi+0.5*h
    
    #define x at 1 step
    x_ipo=xi+h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:]=h*y_derivs[:]
    
    #get k2 values
    y_temp[:]=yi[:]+0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:]=yi[:]+0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:]=yi[:]+0.5*k3[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

# Define an adaptive step size driver for RK4

In [6]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate error by taking one step of size h vs. two steps of size h/2
        y_2  = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1  = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol>1.0):
            
            #decrease the step
            h_step *= SAFETY*(Delta.max()/tol)**(-0.25)
        
        #check iteration
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        
        #iterate
        i+=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9),h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

# Define a wrapper for RK4

In [9]:
def rk4_mv(dfdx,a,b,y_a,tol):
    
    #dfdx is the derivative of x
    #a is upper bound
    #b is lower bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi=a
    yi=y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled ODEs to the size of y_a
    nv=len(y_a)
    
    #set initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag=0
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y=y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
        
        
        #iterate
        i+=1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb =%9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is ==b
        if(xi==b):
            flag=0
    
    #return the answer
    return x, y

In [10]:
a=0.0
b=2.0*np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] =1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

i =   1	x = 0.00062832	h = 0.00062832	b =6.28318531
i =   2	x = 0.00188496	h = 0.00125664	b =6.28318531
i =   3	x = 0.00439823	h = 0.00251327	b =6.28318531
i =   4	x = 0.00886437	h = 0.00446614	b =6.28318531
i =   5	x = 0.01343975	h = 0.00457538	b =6.28318531
i =   6	x = 0.01797124	h = 0.00453149	b =6.28318531
i =   7	x = 0.02252043	h = 0.00454919	b =6.28318531
i =   8	x = 0.02706280	h = 0.00454237	b =6.28318531
i =   9	x = 0.03160823	h = 0.00454543	b =6.28318531
i =  10	x = 0.03615282	h = 0.00454460	b =6.28318531
i =  11	x = 0.04069821	h = 0.00454539	b =6.28318531
i =  12	x = 0.04524381	h = 0.00454560	b =6.28318531
i =  13	x = 0.04978991	h = 0.00454610	b =6.28318531
i =  14	x = 0.05433647	h = 0.00454656	b =6.28318531
i =  15	x = 0.05888357	h = 0.00454710	b =6.28318531
i =  16	x = 0.06343124	h = 0.00454767	b =6.28318531
i =  17	x = 0.06797954	h = 0.00454830	b =6.28318531
i =  18	x = 0.07252851	h = 0.00454897	b =6.28318531
i =  19	x = 0.07707820	h = 0.00454969	b =6.28318531
i =  20	x = 

i = 377	x = 1.78642914	h = 0.00459326	b =6.28318531
i = 378	x = 1.79102467	h = 0.00459553	b =6.28318531
i = 379	x = 1.79562254	h = 0.00459786	b =6.28318531
i = 380	x = 1.80022278	h = 0.00460025	b =6.28318531
i = 381	x = 1.80482547	h = 0.00460268	b =6.28318531
i = 382	x = 1.80943064	h = 0.00460518	b =6.28318531
i = 383	x = 1.81403837	h = 0.00460772	b =6.28318531
i = 384	x = 1.81864869	h = 0.00461032	b =6.28318531
i = 385	x = 1.82326166	h = 0.00461298	b =6.28318531
i = 386	x = 1.82787735	h = 0.00461569	b =6.28318531
i = 387	x = 1.83249580	h = 0.00461845	b =6.28318531
i = 388	x = 1.83711707	h = 0.00462127	b =6.28318531
i = 389	x = 1.84174123	h = 0.00462415	b =6.28318531
i = 390	x = 1.84636831	h = 0.00462709	b =6.28318531
i = 391	x = 1.85099839	h = 0.00463008	b =6.28318531
i = 392	x = 1.85563152	h = 0.00463313	b =6.28318531
i = 393	x = 1.86026775	h = 0.00463624	b =6.28318531
i = 394	x = 1.86490716	h = 0.00463940	b =6.28318531
i = 395	x = 1.86954978	h = 0.00464262	b =6.28318531
i = 396	x = 

i = 756	x = 3.59675504	h = 0.00478493	b =6.28318531
i = 757	x = 3.60154553	h = 0.00479049	b =6.28318531
i = 758	x = 3.60634165	h = 0.00479613	b =6.28318531
i = 759	x = 3.61114351	h = 0.00480185	b =6.28318531
i = 760	x = 3.61595116	h = 0.00480766	b =6.28318531
i = 761	x = 3.62076471	h = 0.00481355	b =6.28318531
i = 762	x = 3.62558424	h = 0.00481953	b =6.28318531
i = 763	x = 3.63040983	h = 0.00482559	b =6.28318531
i = 764	x = 3.63524156	h = 0.00483173	b =6.28318531
i = 765	x = 3.64007953	h = 0.00483797	b =6.28318531
i = 766	x = 3.64492383	h = 0.00484430	b =6.28318531
i = 767	x = 3.64977454	h = 0.00485071	b =6.28318531
i = 768	x = 3.65463176	h = 0.00485722	b =6.28318531
i = 769	x = 3.65949558	h = 0.00486382	b =6.28318531
i = 770	x = 3.66436609	h = 0.00487051	b =6.28318531
i = 771	x = 3.66924339	h = 0.00487730	b =6.28318531
i = 772	x = 3.67412758	h = 0.00488419	b =6.28318531
i = 773	x = 3.67901875	h = 0.00489117	b =6.28318531
i = 774	x = 3.68391699	h = 0.00489825	b =6.28318531
i = 775	x = 

i = 1109	x = 5.28967342	h = 0.00494988	b =6.28318531
i = 1110	x = 5.29463123	h = 0.00495781	b =6.28318531
i = 1111	x = 5.29959708	h = 0.00496585	b =6.28318531
i = 1112	x = 5.30457108	h = 0.00497400	b =6.28318531
i = 1113	x = 5.30955335	h = 0.00498227	b =6.28318531
i = 1114	x = 5.31454400	h = 0.00499066	b =6.28318531
i = 1115	x = 5.31954317	h = 0.00499916	b =6.28318531
i = 1116	x = 5.32455096	h = 0.00500779	b =6.28318531
i = 1117	x = 5.32956749	h = 0.00501654	b =6.28318531
i = 1118	x = 5.33459291	h = 0.00502541	b =6.28318531
i = 1119	x = 5.33962732	h = 0.00503441	b =6.28318531
i = 1120	x = 5.34467087	h = 0.00504354	b =6.28318531
i = 1121	x = 5.34972367	h = 0.00505281	b =6.28318531
i = 1122	x = 5.35478587	h = 0.00506220	b =6.28318531
i = 1123	x = 5.35985760	h = 0.00507173	b =6.28318531
i = 1124	x = 5.36493900	h = 0.00508140	b =6.28318531
i = 1125	x = 5.37003020	h = 0.00509120	b =6.28318531
i = 1126	x = 5.37513135	h = 0.00510115	b =6.28318531
i = 1127	x = 5.38024260	h = 0.00511125	b =6.28